In [127]:
import time
import pandas as pd
import random
import math
import numpy as np
from scipy.spatial import distance_matrix

In [128]:
# Чтение файлов
def file_reader(file_path):
    data = {
        'name': '',
        'track_no': 0,
        'optimal_val': 0,
        'type': '',
        'dimension': 0,
        'capacity': 0,
        'node_coords': {},
        'demands': {},
        'depot': None,
    }
    
    section = None
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('NAME'):
                data['name'] = line.split(':')[1].strip()
            elif line.startswith('COMMENT'):
                comment = line.strip()

                track_no = ''
                for i in range(40, len(comment)):
                    symb = comment[i]
                    if symb == ',':
                        break
                    track_no = track_no + symb
                data['track_no'] = int(track_no)

                optimal_val = ''
                for i in range(2, len(comment)):
                    symb = comment[len(comment)-i]
                    if symb == ' ':
                        break
                    optimal_val = symb + optimal_val
                data['optimal_val'] = int(optimal_val)
                
            elif line.startswith('TYPE'):
                data['type'] = line.split(':')[1].strip()
            elif line.startswith('DIMENSION'):
                data['dimension'] = int(line.split(':')[1].strip())
            elif line.startswith('CAPACITY'):
                data['capacity'] = int(line.split(':')[1].strip())
            elif line == 'NODE_COORD_SECTION':
                section = 'NODE_COORD'
                continue
            elif line == 'DEMAND_SECTION':
                section = 'DEMAND'
                continue
            elif line == 'DEPOT_SECTION':
                section = 'DEPOT'
                continue
            elif line == 'EOF':
                break
                
            if section == 'NODE_COORD':
                parts = line.split()
                if len(parts) >= 3:
                    node_id = int(parts[0])
                    x = int(parts[1])
                    y = int(parts[2])
                    data['node_coords'][node_id] = (x, y)
                    
            elif section == 'DEMAND':
                parts = line.split()
                if len(parts) >= 2:
                    node_id = int(parts[0])
                    demand = int(parts[1])
                    data['demands'][node_id] = demand
                    
            elif section == 'DEPOT':
                if line.strip() == '-1':
                    continue
                depot_id = int(line.strip())
                data['depot'] = depot_id
                
    return data

In [129]:
# Случайные решения
def random_solution(n, m):

    m -= 1 # Так мы будем уверены, что путей не больше, чем машин
    
    total_length = n + m - 1  # Общая длина списка
    
    result = [None] * total_length
    
    remaining_ones = m
    
    # Размещаем единицы так, чтобы они не стояли в соседних ячейках
    available_positions = [i for i in range(1, total_length) if i != 1]
    while remaining_ones > 0 and available_positions:
        pos = random.choice(available_positions)
        result[pos] = 1
        remaining_ones -= 1
        
        neighbors = {pos - 1, pos, pos + 1}
        available_positions = [p for p in available_positions if p not in neighbors]
    
    # Заполняем оставшиеся позиции числами от 2 до n
    numbers = list(range(2, n + 1))
    random.shuffle(numbers)
    
    num_index = 0
    for i in range(total_length):
        if result[i] is None:
            result[i] = numbers[num_index]
            num_index += 1
    
    return result

In [130]:
# Целевая функция
def calc_result(used_solution, capacity, dist_matrix, demands, base_penalty):

    solution = used_solution.copy()

    solution.append(1)
    solution.insert(0, 1)

    z = 0
    demand = 0
    for i in range(1, len(solution)):

        z += dist_matrix[solution[i]-1][solution[i-1]-1]

        if solution[i] == 1:
            if demand > capacity:
                penalty = (demand-capacity)**2 / capacity * base_penalty
            else:
                penalty = 0

            z += penalty
            demand = 0
        else:
            demand += demands[solution[i]] 

    return z

In [131]:
# Мутация
def mutation(prev_solution):
    solution = prev_solution.copy()
    n = len(solution)
    l = max(1, round(n * 0.05))
    
    # Два непересекающихся подсписка
    i = random.randint(0, n - l)
    j = random.randint(0, n - l)
    
    while abs(i - j) < l:
        j = random.randint(0, n - l)
    
    sublist1 = solution[i:i+l]
    sublist2 = solution[j:j+l]
    
    # С вероятностью 30% отзеркаливаем один из подсписков
    if random.random() < 0.3:
        if random.choice([True, False]):
            sublist1 = sublist1[::-1]
        else:
            sublist2 = sublist2[::-1]
    
    solution[i:i+l] = sublist2
    solution[j:j+l] = sublist1
    
    return solution

In [132]:
def process_files(file_list, folder, results_file, solutions_num, base_penalty):

    df = pd.read_csv(results_file)
    
    for file_name in file_list:
        data = file_reader(f'{folder}\\{file_name}')

        # Матрица расстояний и общий путь
        coords = np.array([data['node_coords'][node_id] for node_id in sorted(data['node_coords'])])
        dist_matrix = distance_matrix(coords, coords)
        total_path = sum(sum(row) for row in dist_matrix) // 2

        start = time.time()

        print(f'===== Обработка {file_name} ================')

        solutions = []
        z_s = []
        
        print('Первые решения:')
        for i in range(solutions_num):
            solutions.append(random_solution(data['dimension'], data['track_no']))
            z = calc_result(solutions[i], data['capacity'], dist_matrix, data['demands'], base_penalty)
            z_s.append(z)
            print(f'{round(z)}: {solutions[i]}')


        ############################################################

        work_time = round(time.time() - start, 6)
        
        idx = df.index[df['name'] == data['name']].tolist()
        
        df.loc[idx[0], 'time'] = work_time
        df.loc[idx[0], 'value'] = '10000000'

        # print(f'Обработка завершена\n')
        # print(f'Время: {work_time}')
        # print(f'---------------------------------------------\n\n')
    
    df.to_csv(f'{folder}_results\{results_file}', index=False)

In [133]:
# Названия файлов
A = [
    "A-n32-k5.vrp", "A-n33-k5.vrp", "A-n33-k6.vrp",
    "A-n34-k5.vrp", "A-n36-k5.vrp", "A-n37-k5.vrp",
    "A-n37-k6.vrp", "A-n38-k5.vrp", "A-n39-k5.vrp",
    "A-n39-k6.vrp", "A-n44-k6.vrp", "A-n45-k6.vrp",
    "A-n45-k7.vrp", "A-n46-k7.vrp", "A-n48-k7.vrp",
    "A-n53-k7.vrp", "A-n54-k7.vrp", "A-n55-k9.vrp",
    "A-n60-k9.vrp", "A-n61-k9.vrp", "A-n62-k8.vrp",
    "A-n63-k9.vrp", "A-n63-k10.vrp", "A-n64-k9.vrp",
    "A-n65-k9.vrp", "A-n69-k9.vrp", "A-n80-k10.vrp"
]

B = [
    "B-n31-k5.vrp", "B-n34-k5.vrp", "B-n35-k5.vrp",
    "B-n38-k6.vrp", "B-n39-k5.vrp", "B-n41-k6.vrp",
    "B-n43-k6.vrp", "B-n44-k7.vrp", "B-n45-k5.vrp",
    "B-n45-k6.vrp", "B-n50-k7.vrp", "B-n50-k8.vrp",
    "B-n51-k7.vrp", "B-n52-k7.vrp", "B-n56-k7.vrp",
    "B-n57-k7.vrp", "B-n57-k9.vrp", "B-n63-k10.vrp",
    "B-n64-k9.vrp", "B-n66-k9.vrp", "B-n67-k10.vrp",
    "B-n68-k9.vrp", "B-n78-k10.vrp"
]

In [134]:
solutions_num = 3
base_penalty = 50

# Обработка папки "А"
process_files(A, 'A', 'A_results.csv', solutions_num, base_penalty)

# Обработка папки "B"
process_files(B, 'B', 'B_results.csv', solutions_num, base_penalty)

===== Обработка A-n32-k5.vrp ================
Первые решения:
3405: [18, 30, 17, 1, 22, 25, 6, 1, 24, 23, 27, 4, 7, 26, 5, 15, 8, 1, 10, 21, 11, 2, 32, 20, 1, 12, 16, 14, 29, 9, 3, 28, 13, 19, 31]
11893: [15, 14, 23, 31, 18, 32, 7, 20, 9, 30, 25, 22, 16, 10, 13, 5, 24, 21, 1, 12, 2, 19, 29, 17, 1, 3, 6, 1, 27, 26, 8, 11, 28, 1, 4]
5003: [27, 12, 31, 13, 29, 19, 1, 17, 15, 5, 26, 24, 1, 30, 11, 16, 25, 1, 32, 14, 4, 6, 1, 2, 21, 10, 18, 8, 9, 20, 7, 23, 28, 3, 22]
===== Обработка A-n33-k5.vrp ================
Первые решения:
25988: [4, 9, 2, 13, 1, 10, 1, 32, 17, 24, 1, 25, 3, 30, 33, 26, 19, 18, 31, 23, 21, 15, 27, 8, 7, 16, 11, 14, 12, 20, 29, 5, 22, 6, 1, 28]
17642: [10, 18, 14, 33, 24, 1, 27, 1, 30, 11, 1, 3, 6, 19, 12, 15, 31, 23, 22, 32, 9, 21, 20, 5, 4, 28, 7, 26, 8, 13, 2, 1, 29, 17, 25, 16]
2441: [9, 3, 7, 33, 11, 19, 24, 22, 1, 26, 15, 27, 12, 21, 2, 13, 1, 30, 23, 32, 17, 28, 1, 16, 29, 14, 4, 1, 5, 10, 6, 8, 31, 20, 25, 18]
===== Обработка A-n33-k6.vrp ================
Первы

C:\Users\} { ome\AppData\Local\Temp\ipykernel_23980\255741875.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx[0], 'time'] = work_time
C:\Users\} { ome\AppData\Local\Temp\ipykernel_23980\255741875.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10000000' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx[0], 'value'] = '10000000'
C:\Users\} { ome\AppData\Local\Temp\ipykernel_23980\255741875.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx[0], 'time'] = work_time
C:\Users